ARCHIVO XML

Este es el fichero con el trabajo individualizado para el tratamiento e inserción de datos de archivo xml.
Los procesos aquí implentados se trasladarán a los métodos que se definan en la clase de tratamiento de la base de datos. 


In [ ]:
import re
import os
import xml.etree.ElementTree as ET

tree = ET.parse('data_xml.xml')
root = tree.getroot()

xml_list = []
for child in root:
    xml_dict = {}
    for subchild in child:
        xml_dict[subchild.tag] = subchild.text
    xml_list.append(xml_dict) # Es una lista que contiene diccionarios


xml_list

In [ ]:
#Creamos una copia para comenzar con los diferentes procesos de limpieza del archivo
limpieza_xml = xml_list.copy()
limpieza_xml

In [ ]:
#eliminamos la columna level_0 al estimar que no es necesaria
for i in limpieza_xml:
    if "level_0" in i:
        i.pop("level_0")    
    else:
        pass 

In [ ]:
# realizamos la modificación de la columna gender para cambiar los atributos numéricos por palabras y ampliar el abanico de opciones


for i in limpieza_xml:
    if "gender" in i:
        if  i["gender"] == "0":
            i["gender"] = "Man"
       
        elif i["gender"] == "1":
            i["gender"] = "Woman"
        
        elif i["gender"] == "2":
            i["gender"] = "Nonbinary"    

        elif i["gender"] == "3":
            i["gender"] = "Prefer not to say"    

        elif i["gender"] == "4":
            i["gender"] = "Prefer to self-describe"

    else:
        pass        


limpieza_xml        

In [ ]:
#sacamos los values del diccionario que son los valores a insertar en una lista de tuplas
lista_tuplas = []
for index in range(len(limpieza_xml)):
    value = limpieza_xml[index].values()
    lista_tuplas.append(value)

In [ ]:
#comprobamos la lista de tuplas
print(lista_tuplas[0:5])


In [ ]:
# creamos una lista para extraer solo los valores. Será una lista de tuplas que ya tendrá un formato válido para el insert
limpieza_xml_values = []
for i in (lista_tuplas):
    val = tuple(i)
    limpieza_xml_values.append(val)
 

#print ((limpieza_xml[0]).values())


In [ ]:
limpieza_xml_values

In [ ]:
#realizamos la inserción 
import mysql.connector

#el método debería pedir la tabla

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='prueba_proyecto')


mycursor = cnx.cursor()
sql = "INSERT INTO tablaxml (index_sql, time, age, gender) VALUES (%s, %s, %s, %s)" 

val = limpieza_xml_values

try: 
    mycursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registros insertados.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)